# Main Draft 

In [18]:
using Plots
using Pandas
using PyCall
np = pyimport("numpy")

importall Absopac_J #.py files, not notebooks, dev in notebooks, but only import as .py
#using .Setup_Atm_J #import *
#using integrate_Transits_J #import *

#from aerosols import *
#from vis import *

# ----------------------------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import time
# ----------------------------------------------------------------------------------------------
from absopac import *
#from aerosols import *
from integrate_transits import *
from setup_atm import *
#from vis import *

LoadError: UndefVarError: importall not defined

In [4]:
function tspec_from_dict(atmdict)
    #=
    Input dictionary should have 
    wlrange in microns
    beta in degrees
    Z in terms of Zsolar
    everything else with dimensions in SI units
    =#

    nalt = atmdict['nalt']   # number of altitude grid layers
    nlong = atmdict['nlong'] # number of longitude grid points
    nlat = atmdict['nlat']   # number of latitude grid points
    atm_type = atmdict['atm_type'] # should be one of: clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d    
    
    P0 = atmdict['P0']  # reference pressure in Pascals
    Z = atmdict['Z']    # metallicity in multiples of Z_sun
    Mp = atmdict['Mp']  # Mass of the planet in kg
    R0 = atmdict['Rp']  # reference radius for planet where it is opaque in meters
    Rstar = atmdict['Rstar'] # radius of star in meters
    wlrange = atmdict['wlrange'] # wavelength array for spectrum, in microns
    nwl = len(wlrange)

    function gasOpacFunc(Z,T,P) #based on metallicity,
        # expects T in Kelvin, P in pascals
        # expects T and P to be 1d arrays
        # of equal length
        mu = mu_func(Z,T,P)
        rho = eos_idealgas(P, mu*gmol_to_kg, T)
        gas = gasopac_func(Z, wlrange, T, rho)
        rayleigh = rayleigh_func(Z, wlrange, T, rho)
        return gas + rayleigh # units are light blocked per m, has shape (nT, nwl) or (nwl,) if nz=1
    end 
    
    if atm_type == 'clear1d':
        Temp = atmdict['Temp']
        solved_atm = atm_setup_1d(Temp,Mp,R0,P0,nwl,nalt,nlong,nlat,mu_func,Z) #atmosphere set up
        Temps, Pressures, alt_range, long_range, lat_range, atmosphere_grid = solved_atm
        true_depth = effective_area_at_full_depth_no_long_dep(alt_range, long_range, 
                    lat_range, Temps, Pressures, gasOpacFunc,nwl,100,Z)/(np.pi*Rstar**2.0)
        
    else:
        print('Unknown atmosphere type, please pick one of: \n') 
        print('clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d')
        return None
    end 
    return true_depth
end

NameError: name 'alt_range' is not defined

In [6]:
atmdict = Dict{ "wlrange": np.logspace(np.log10(0.5),np.log10(12),300), # wavelength array for spectrum, in microns
            "nalt": 100, # number of altitude grid layers
            "nlong":60, # number of longitude grid points
            "nlat":60,  # number of latitude grid points
 
            "P0":10^5, # reference pressure in Pascals
            "Z":1.0, # metallicity in multiples of Z_sun
            "Mp": 0.8*Mj, # mass of the planet in kg
            "Rp": 1.5*Rj,  # reference radius for the planet where all wavelengths are opaque, in meters
            "Rstar": 0.9*Rsun, # radius of host star in meters    
           
            "atm_type" :"clear1d", # should be one of: clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d    
            "Temp":1600, # isothermal temperature
        }
clear1d_tspec = tspec_from_dict(atmdict)
plt.semilogx(atmdict["wlrange"],clear1d_tspec*100)
plt.xlabel("Wavelength ($\mu$m)",fontsize=18)
plt.ylabel("Transit Depth (%)",fontsize=18) 

''